### This script allows to merge the data from Johns Hopkins and Yatko APIs 

In [60]:
import numpy as np
import pandas as pd
import requests
import os
import csv 
import warnings
warnings.filterwarnings("ignore")

In [61]:
df = pd.read_csv('8thJuly_CLEANED_covid_summary_dump.csv') #taking in the summary data from John Hopkinns API
df_2 = pd.read_csv(r'C:\Users\apurv\Data Science in Python\Project\Datascripts\CleanData_YatkoAPI.csv') #fetching the Yatko_Cleaned Data

In [62]:
df_2.head()

,Country,CountryCode,total_cases,active_cases,deaths,recovered,critical,tested,death_ratio,recovery_ratio
0,USA,US,3097538,1608023,133991,1355524,1608023,38806121,0.043257,0.437613
1,Brazil,BR,1674655,489865,66868,1117922,489865,4359978,0.039929,0.667554
2,India,IN,746506,267204,20684,458618,267204,10473771,0.027708,0.614353
3,Russia,RU,700792,217614,10667,472511,217614,21790705,0.015221,0.674253
4,Peru,PE,309278,97388,10952,200938,97388,1821328,0.035412,0.649700


In [63]:
#Function to make the Yatko data feasible to integrate with Johns Hopkinns data
#Pre-processing on Yatko 

def Data_integration(df,df2):
    #finding out the countries where null values exist using the CountryCode
    U = df[df['CountryCode'].isnull()].index.tolist()
    df.loc[U,'CountryCode'] = "NA" 
    #renaming some records to match the two dfs easily 
    df2 = df2.replace(to_replace ="USA", value ="United States of America") #replacing USA and UK with their complete names
    df2 = df2.replace(to_replace ="UK", value ="United Kingdom")
    K = df2[df2['CountryCode'].isnull()].index.tolist()
    for i in range(len(K)-1):
        L = df2.loc[K[i],"Country"] #popping countries not required
        if (L == 'Diamond Princess'):
            K.pop(i)
        elif (L == 'Ms Zaandam'):
            K.pop(i)
    
    #handling data for the countries Congo, CAR (Central African Republic), Namibia, Lesotho to match in both the dfs
    
    for i in range(len(K)):
        E = df2.loc[K[i],'Country']
        Y = df[df['Country']==E].index.values.astype(int)
        if (E == 'DRC'):
            df2.loc[K[i],'Country'] = 'Congo (Kinshasa)'
        elif (E == 'Car'):
            df2.loc[K[i],'Country'] = 'Central African Republic'
    
    for i in range(len(K)):
        N = df2.loc[K[i],'Country']
        M = df[df['Country']==N].index.values.astype(int)
        if (N == 'Congo (Kinshasa)'):
            df2.loc[K[i],'CountryCode'] = df.loc[M[0],'CountryCode']
        elif (N == 'Central African Republic'):
            df2.loc[K[i],'CountryCode'] = df.loc[M[0],'CountryCode']
        elif (N == 'Namibia'):
            df2.loc[K[i],'CountryCode'] = df.loc[M[0],'CountryCode']
        elif (N == 'Lesotho'):
            df2.loc[K[i],'CountryCode'] = df.loc[M[0],'CountryCode']
        
    J = df2[df2['CountryCode'].isnull()].index.tolist()
    for i in range(len(J)):
        df2 = df2.drop(J[i], axis = 'index')
    
    df2 = df2.sort_values(by = ['Country'], ascending = True) #sorting the df Country wise
    
    return df2

    

In [64]:
Yatko_summary = Data_integration(df, df_2)

In [65]:
Yatko_summary.head()

,Country,CountryCode,total_cases,active_cases,deaths,recovered,critical,tested,death_ratio,recovery_ratio
43,Afghanistan,AF,33594,12353,936,20305,12353,77501,0.027862,0.604423
104,Albania,AL,3106,1232,83,1791,1232,26637,0.026722,0.576626
58,Algeria,DZ,16879,3817,968,12094,3817,0,0.057349,0.716512
148,Andorra,AD,855,3,52,800,3,3750,0.060819,0.935673
162,Angola,AO,386,248,21,117,248,10000,0.054404,0.303109


In [42]:
#Yatko_summary.to_csv("Yatko_summary_test.csv",index = False)

In [66]:
#function to merge both the dataframes

def Merge_data(df,df2):
    M_df = pd.merge(df, df2, on = ["CountryCode"], how = "inner") #using inner join on the two dfs to retain all values
    M_df = M_df.drop_duplicates('CountryCode', keep = "first") #deleting the rest duplicates(if any),only keeping first 
    TP  = df.iloc[[df[df['CountryCode']=='XK'].index.values.astype(int)[0]]] #Handling data for country with countrycode 'XK'
    TP = TP.rename(columns= {'Country': 'Country_x'})
    M_df = M_df.append(TP)
    M_df = M_df.fillna(0)
    M_df = M_df.drop(['Country_y', 'Unnamed: 0', 'active_cases', 'critical', 'deaths', 'recovered','total_cases'], axis = 1)
    M_df = M_df.rename(columns= {'Country_x': 'Country','death_ratio': 'DeathRatio','recovery_ratio': 'RecoveryRatio','tested': 'Total Tests Done'})
    M_df = M_df.sort_values(by=['Country'], ascending = True)
    return M_df

In [67]:
Merged_data = Merge_data(df,Yatko_summary)

In [68]:
Merged_data.head()

,CountryCode,Country,Date,NewConfirmed,NewDeaths,NewRecovered,TotalConfirmed,TotalDeaths,TotalRecovered,DeathRatio,RecoveryRatio,Total Tests Done
0,AF,Afghanistan,2020-07-08,194,22,76,33384,920,20179,0.027862,0.604423,77501.0
1,AL,Albania,2020-07-08,74,2,42,3038,81,1744,0.026722,0.576626,26637.0
2,DZ,Algeria,2020-07-08,475,9,210,16879,968,12094,0.057349,0.716512,0.0
3,AD,Andorra,2020-07-08,0,0,0,855,52,800,0.060819,0.935673,3750.0
4,AO,Angola,2020-07-08,40,2,9,386,21,117,0.054404,0.303109,10000.0


In [46]:
#TR = list(Merged_data['CountryCode'])
#TE = list(df['CountryCode'])
#SR = list(set(TR) - set(TE))
#SR

In [69]:
Merged_data.to_csv("MergedDF_Yatko_JohnsHopkinns.csv", index = False)